<a href="https://colab.research.google.com/github/unajmieh/Log-clickstream-Analysis/blob/main/Apache_Spark_Log_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

df = pd.read_csv('./Apache_2k.log_structured.csv')
df.head()


,LineId,Time,Level,Content,EventId,EventTemplate
0,1,Sun Dec 04 04:47:44 2005,notice,workerEnv.init() ok /etc/httpd/conf/workers2.p...,E2,workerEnv.init() ok <*>
1,2,Sun Dec 04 04:47:44 2005,error,mod_jk child workerEnv in error state 6,E3,mod_jk child workerEnv in error state <*>
2,3,Sun Dec 04 04:51:08 2005,notice,jk2_init() Found child 6725 in scoreboard slot 10,E1,jk2_init() Found child <*> in scoreboard slot <*>
3,4,Sun Dec 04 04:51:09 2005,notice,jk2_init() Found child 6726 in scoreboard slot 8,E1,jk2_init() Found child <*> in scoreboard slot <*>
4,5,Sun Dec 04 04:51:09 2005,notice,jk2_init() Found child 6728 in scoreboard slot 6,E1,jk2_init() Found child <*> in scoreboard slot <*>


In [8]:
import re

def clean_content(content):
    cleaned = re.sub(r'/[^ ]+', '', content)  # Remove anything starting with a '/' until a space
    cleaned = re.sub(r'\b(\d+)\b', '<ID>', cleaned)  # Replace standalone numbers with <ID>
    cleaned = re.sub(r'(\b[A-Za-z_][A-Za-z0-9_]*)(\(\))', r'\1', cleaned)  # Keep function names without parentheses
    cleaned = re.sub(r'\b(ok|state|in|slot|error)\b', '', cleaned, flags=re.IGNORECASE)  # Remove unimportant terms
    cleaned = re.sub(r'\s+', ' ', cleaned)  # Replace multiple spaces with a single space
    cleaned = cleaned.strip()  # Strip leading and trailing whitespace
    return cleaned

df['Cleaned_Content'] = df['Content'].apply(clean_content)
print(df[['LineId', 'Cleaned_Content']])


      LineId                            Cleaned_Content
0          1                             workerEnv.init
1          2                mod_jk child workerEnv <ID>
2          3  jk2_init Found child <ID> scoreboard <ID>
3          4  jk2_init Found child <ID> scoreboard <ID>
4          5  jk2_init Found child <ID> scoreboard <ID>
...      ...                                        ...
1995    1996                mod_jk child workerEnv <ID>
1996    1997  jk2_init Found child <ID> scoreboard <ID>
1997    1998  jk2_init Found child <ID> scoreboard <ID>
1998    1999                             workerEnv.init
1999    2000                mod_jk child workerEnv <ID>

[2000 rows x 2 columns]


In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Cleaned_Content_Encoded'] = label_encoder.fit_transform(df['Cleaned_Content'])
#print(df[['LineId', 'Cleaned_Content', 'Cleaned_Content_Encoded']])


In [12]:
def clean_event_template(template):
    # Remove placeholders like <*>
    cleaned = re.sub(r'<\*\>', '', template)

    # Replace numeric identifiers with <ID>
    cleaned = re.sub(r'\b\d+\b', '<ID>', cleaned)

    # Remove specific unwanted keywords while preserving function/command patterns
    cleaned = re.sub(r'\b(state)\b', '', cleaned, flags=re.IGNORECASE)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()  # Normalize whitespace

    return cleaned

# Apply the cleaning function to the EventTemplate column
df['Cleaned_EventTemplate'] = df['EventTemplate'].apply(clean_event_template)

# Show the resulting DataFrame
print(df[['LineId', 'EventTemplate', 'Cleaned_EventTemplate']])

      LineId                                      EventTemplate  \
0          1                            workerEnv.init() ok <*>   
1          2          mod_jk child workerEnv in error state <*>   
2          3  jk2_init() Found child <*> in scoreboard slot <*>   
3          4  jk2_init() Found child <*> in scoreboard slot <*>   
4          5  jk2_init() Found child <*> in scoreboard slot <*>   
...      ...                                                ...   
1995    1996          mod_jk child workerEnv in error state <*>   
1996    1997  jk2_init() Found child <*> in scoreboard slot <*>   
1997    1998  jk2_init() Found child <*> in scoreboard slot <*>   
1998    1999                            workerEnv.init() ok <*>   
1999    2000          mod_jk child workerEnv in error state <*>   

                          Cleaned_EventTemplate  
0                           workerEnv.init() ok  
1               mod_jk child workerEnv in error  
2     jk2_init() Found child in scoreboard sl

In [13]:
# Apply the cleaning function to the EventTemplate column
df['Cleaned_EventTemplate'] = df['EventTemplate'].apply(clean_event_template)

# Create LabelEncoder instance
label_encoder = LabelEncoder()

# Label encode the Cleaned_EventTemplate
df['Cleaned_EventTemplate_Encoded'] = label_encoder.fit_transform(df['Cleaned_EventTemplate'])

# Show the resulting DataFrame
print(df[['LineId', 'EventTemplate', 'Cleaned_EventTemplate', 'Cleaned_EventTemplate_Encoded']])

      LineId                                      EventTemplate  \
0          1                            workerEnv.init() ok <*>   
1          2          mod_jk child workerEnv in error state <*>   
2          3  jk2_init() Found child <*> in scoreboard slot <*>   
3          4  jk2_init() Found child <*> in scoreboard slot <*>   
4          5  jk2_init() Found child <*> in scoreboard slot <*>   
...      ...                                                ...   
1995    1996          mod_jk child workerEnv in error state <*>   
1996    1997  jk2_init() Found child <*> in scoreboard slot <*>   
1997    1998  jk2_init() Found child <*> in scoreboard slot <*>   
1998    1999                            workerEnv.init() ok <*>   
1999    2000          mod_jk child workerEnv in error state <*>   

                          Cleaned_EventTemplate  Cleaned_EventTemplate_Encoded  
0                           workerEnv.init() ok                              5  
1               mod_jk child work

In [ ]:
# have a timeseries plot
# the target is the level
# find large models for classification task

In [ ]:
!pip install transformers==4.31.0

from transformers import XLNetForSequenceClassification, XLNetTokenizer

model_name = "xlnet-base-cased"  # Choose an XLNet variant
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=your_num_labels)


In [ ]:
!pip install transformers==4.31.0

from transformers import ElectraForSequenceClassification, ElectraTokenizer

model_name = "google/electra-base-discriminator"  # Choose an ELECTRA variant
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=your_num_labels)

In [ ]:
!pip install transformers==4.31.0

from transformers import DebertaForSequenceClassification, DebertaTokenizer

model_name = "microsoft/deberta-base"  # Choose a DeBERTa variant
tokenizer = DebertaTokenizer.from_pretrained(model_name)
model = DebertaForSequenceClassification.from_pretrained(model_name, num_labels=your_num_labels)

In [ ]:
!pip install transformers==4.31.0

from transformers import LongformerForSequenceClassification, LongformerTokenizer

model_name = "allenai/longformer-base-4096"  # Choose a Longformer variant
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerForSequenceClassification.from_pretrained(model_name, num_labels=your_num_labels)


# Apache Spark Models

In [ ]:
from pyspark.ml.classification import GBTClassifier

# Assuming 'df' is your Spark DataFrame with features and label column
gbt = GBTClassifier(featuresCol='features', labelCol='label')
gbt_model = gbt.fit(df)

# Make predictions
predictions = gbt_model.transform(df)


Aggregations and Reporting
Use Spark SQL for complex aggregations and groupBy operations.
Generate detailed reports and visualizations (using tools like Apache Zeppelin or Databricks) on system performance metrics, such as the number of errors over time, types of warnings logged, etc.
7. Machine Learning for Predictive Maintenance
Gather patterns from logs indicating when services have likely failed or are about to fail.
Train predictive models using past logs to forecast system failures based on historical patterns.
8. Distributed Machine Learning
Use the Spark MLlib to perform distributed machine learning tasks, such as clustering (e.g., K-means) to group similar log entries or classification tasks (e.g., predicting log types) based on historical trends.
9. Data Enrichment
Enrich the log data by joining it with other datasets (such as user databases or configuration files) to provide additional context to the events recorded in the logs.
Perform ETL (Extract, Transform, Load) processes to prepare data for reporting or analysis.
10. Graph Analysis for Interactions
If the data involves interactions (e.g., user sessions), create graphs to analyze relationships and interactions between different log events.
Use Spark GraphX to identify clusters or important points in interaction networks.